In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as skl
import anndata as ann
import random, os
from scipy.stats import pearsonr as pr
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score as f1
from sklearn.metrics import precision_recall_curve as prc
from sklearn.metrics import silhouette_score as sil
from sklearn.metrics import auc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_score, recall_score, average_precision_score
from sklearn.metrics import silhouette_score
from torch_geometric.nn import TransformerConv
from torch_geometric.data import Data
import gc, psutil

In [2]:
import torch
import torch_geometric
from torch_geometric.nn import GraphConv, GCNConv
from sklearn.model_selection import StratifiedKFold
import scanpy as sc
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
import torch_geometric.nn as pyg_nn
from torch_geometric.data import Data
from torch_geometric.utils import from_scipy_sparse_matrix
from sklearn.preprocessing import LabelEncoder
from torch_geometric.loader import NeighborLoader


In [3]:
sc.set_figure_params(dpi=200)

In [4]:
import torch
from torch_geometric.data import DataLoader
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import DataLoader
from tqdm import tqdm  # For progress bar
from sklearn.model_selection import KFold

# Set random seeds for reproducibility
import random
import numpy as np
import torch

def set_random_seeds(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # if using CUDA
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_random_seeds(42)
# Check if GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from torch.nn import Linear, Dropout
from torch_geometric.nn import TransformerConv
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score, accuracy_score


In [6]:
from torch_geometric.data import Data
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

In [7]:
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score,
    average_precision_score, confusion_matrix, classification_report
)
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from torch_geometric.loader import RandomNodeLoader
from torch_geometric.utils import subgraph

In [8]:
def memory_usgae():
    gc.collect()
    torch.cuda.empty_cache()
    process = psutil.Process(os.getpid())
    memory_gb = process.memory_info().rss / 1024**3  # in GB

    print(f"Current memory usage: {memory_gb:.2f} GB")

# Load data

In [9]:
# load the data
ad = sc.read_h5ad('./Data/Cancer_cell_data/All_integrated.hallmark.harmony.h5ad')
ad

AnnData object with n_obs × n_vars = 725643 × 2132
    obs: 'Project_ID', 'Primary_or_Metastatic', 'Final_cancer_type', 'Final_histological_subtype', 'Final_molecular_subtype', 'Final_tissue', 'Final_sample_id', 'Final_patient_age', 'Final_patient_stage', 'Final_patient_treatment', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'Final_histological_subtype_backup', 'Final_patient_age_backup', 'Final_tissue_backup', 'Final_patient_treatment_backup', 'Final_patient_stage_backup', 'Classifier_label'
    var: 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
    uns: 'Final_cancer_type_colors', 'Final_histological_subtype_colors', 'Final_molecular_subtype_colors', 'Final_patient_stage_colors', 'Final_patient_treatment_colors', 'Final_tissue_colors', 'Primary_or_Metastatic_colors', 'Project_ID_colors', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_pca_harmony_Project_ID', 'X_pca_harmony_project_id', 'X_uma

In [10]:
# Map cancer type → its "primary tissue"
primary_tissue_map = {
    "Breast Cancer": "Breast",
    "Lung Cancer": "Lung",
    "Ovarian Cancer": "Ovary",
    "Colorectal Cancer": "Colon"
}

# Initialize as primary
ad.obs["source"] = "Primary"


# Loop through rows
for idx, row in ad.obs.iterrows():
    cancer_type = row["Final_cancer_type"]
    tissue = row["Final_tissue"]

    primary_site = primary_tissue_map.get(cancer_type, None)

    if row["Primary_or_Metastatic"] == "Primary":
        ad.obs.at[idx, "source"] = "Primary"

    else:  # Metastatic
        if tissue == primary_site:
            ad.obs.at[idx, "source"] = "local"
        else:
            ad.obs.at[idx, "source"] = "distant"

print(ad.obs["source"].value_counts())


source
Primary    328282
distant    249125
local      148236
Name: count, dtype: int64


In [11]:
memory_usgae()

Current memory usage: 21.06 GB


In [12]:
ad.raw = None


In [13]:
memory_usgae()

Current memory usage: 7.21 GB


# Models

In [14]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier

models = {
    "LogisticRegression": LogisticRegression(max_iter=1000, n_jobs=-1, solver='saga'),
    "SGD": SGDClassifier(loss="log_loss", max_iter=1000, n_jobs=-1),
    "RandomForest": RandomForestClassifier(n_jobs=-1, random_state=42)
    
}


In [15]:
supports_sparse = {
    "SGD": True,
    "LogisticRegression": True,  # only with solver='saga'
    "RandomForest": False,       # convert to dense
}


# LOOCV

## Data preperation

In [16]:
X = ad.X
X

<725643x2132 sparse matrix of type '<class 'numpy.float64'>'
	with 479905723 stored elements in Compressed Sparse Row format>

In [17]:
import torch
import numpy as np

# Labels
primary_mask = (ad.obs["Primary_or_Metastatic"] == "Primary").values
local_mask = (ad.obs["Primary_or_Metastatic"] == "Metastatic") & (ad.obs["source"] == "local").values
distant_mask = (ad.obs["Primary_or_Metastatic"] == "Metastatic") & (ad.obs["source"] == "distant").values

y_3class = np.full(X.shape[0], -1, dtype=int)
y_3class[primary_mask] = 0
y_3class[local_mask] = 1
y_3class[distant_mask] = 2

y_binary_np = np.full(X.shape[0], -1, dtype=int)
y_binary_np[primary_mask] = 0
y_binary_np[local_mask] = 1

# Project IDs (used for LOOCV)
project_ids = ad.obs["Project_ID"].values



In [18]:
# Unique project IDs
unique_projects = np.unique(project_ids)
print(f"Found {len(unique_projects)} unique projects.")

# Build LOOCV folds: exclude distant mets (class 2) from validation
loocv_folds = []
for proj in unique_projects:
    val_mask = (project_ids == proj) & ((y_3class == 0) | (y_3class == 1))  # Only Primary or Local in the project
    train_mask = project_ids != proj  # All others including distant mets

    val_idx = np.where(val_mask)[0]
    
    train_idx = np.where(train_mask)[0]

    if len(val_idx) == 0:
        print(f"Skipping project {proj}: no valid primary/local samples.")
        continue
        
    # print(proj, 'Training', len(train_idx), 'Validation', len(val_idx))

    loocv_folds.append({
        "project": proj,
        "train_idx": train_idx,
        "val_idx": val_idx
    })


Found 27 unique projects.


In [ ]:
# del ad

In [19]:
# save the memory 
memory_usgae()

Current memory usage: 7.31 GB


## Run Models

In [46]:
import joblib
import os

def evaluate_baseline_loocv(model, X_sparse, y_3class, y_binary, loocv_folds, model_name, save_dir_base):
    os.makedirs(save_dir_base, exist_ok=True)
    accs, f1s, aucs, auprcs = [], [], [], []

    use_sparse = supports_sparse.get(model_name, False)
    print(f"Model {model_name} | Sparse: {use_sparse}")
    # Create output directory
    os.makedirs("trained_models", exist_ok=True)

    for fold in loocv_folds:
        
        memory_usgae()
        proj = fold["project"]
        train_idx = fold["train_idx"]
        val_idx = fold["val_idx"]
        
        safe_proj = str(proj).replace("/", "_").replace(" ", "_")
        save_path = f"{save_dir_base}/{model_name}_project_{safe_proj}.joblib"
        if os.path.exists(save_path):
            print(proj, 'already processed, conitnue...')
            continue

        

        # Select subset
        if use_sparse:
            X_train = X_sparse[train_idx]
            X_val = X_sparse[val_idx]
        else:
            X_train = X_sparse[train_idx].toarray()
            X_val = X_sparse[val_idx].toarray()

        y_train = y_3class[train_idx]
        y_val_3class = y_3class[val_idx]
        y_val_binary = y_binary[val_idx]

        model.fit(X_train, y_train)

        # Save model
        safe_proj = str(proj).replace("/", "_").replace(" ", "_")
        save_path = f"{save_dir_base}/{model_name}_project_{safe_proj}.joblib"
        joblib.dump(model, save_path)
        print(f"Saved model to {save_path}")

        # Predict
        y_pred_all = model.predict(X_val)
        y_proba_all = model.predict_proba(X_val) if hasattr(model, "predict_proba") else None

        # Binary eval: Primary vs Local only
        mask = (y_val_binary != -1) & (y_val_3class != 2)
        if mask.sum() == 0:
            print(f"Skipping project {proj}: no Primary vs Local samples in val set.")
            continue

        y_true = y_val_binary[mask]
        y_pred = y_pred_all[mask]
        y_prob = y_proba_all[mask, 1] if y_proba_all is not None else y_pred

        acc = accuracy_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred, average='weighted')
        try:
            auc = roc_auc_score(y_true, y_prob, multi_class='ovr', average='weighted')
        except:
            auc = np.nan
        try:
            auprc = average_precision_score(y_true, y_prob, average='weighted')
        except:
            auprc = np.nan

        accs.append(acc)
        f1s.append(f1)
        aucs.append(auc)
        auprcs.append(auprc)

        print(f"Project {proj}: ACC={acc:.4f}, F1={f1:.4f}, AUC={auc:.4f}, AUPRC={auprc:.4f}")

    # Summary
    print("\nLOOCV Overall Performance (Primary vs Local):")
    print(f"ACC:   {np.mean(accs):.4f} ± {np.std(accs):.4f}")
    print(f"F1:    {np.mean(f1s):.4f} ± {np.std(f1s):.4f}")
    print(f"AUC:   {np.mean(aucs):.4f} ± {np.std(aucs):.4f}")
    print(f"AUPRC: {np.mean(auprcs):.4f} ± {np.std(auprcs):.4f}")

    return {
        "acc": accs,
        "f1": f1s,
        "auc": aucs,
        "auprc": auprcs
    }


In [ ]:
all_loocv_results = dict()
for name, model in models.items():
    print(f"LOOCV: {name}")
    results = evaluate_baseline_loocv(
        model=model,
        X_sparse=X,
        y_3class=y_3class,
        y_binary=y_binary_np,
        loocv_folds=loocv_folds,
        model_name=name,
        save_dir_base= '/scratch/gilbreth/wang3712/Metastasis_single_cell/Baseline_LOOCV/'
    )
    all_loocv_results[name] = results


## Evaluation

In [23]:
project_ids = ad.obs["Project_ID"].values

In [50]:
import joblib


all_results_ad = dict()
cancer_projects =  np.unique(ad.obs['Project_ID'])
cancer_projects
for model in models.keys():
    save_base_dir = '/scratch/gilbreth/wang3712/Metastasis_single_cell/Baseline_LOOCV/'
    all_results_ad[model] = dict()
    print("*" * 15, model, "*" * 15)

    metrics_df = []

    for project in cancer_projects:
        # Your val mask code...
        # Only primary & local mets
        project_mask = (project_ids == project)
        val_mask = (project_ids == project) & ((y_3class == 0) | (y_3class == 1))
        val_mask_idx = np.where(val_mask)[0]

        cancer = ad[ad.obs.Project_ID == project].obs['Final_cancer_type'].value_counts().index[0].split(' ')[0]
        
        all_results_ad[model][cancer+' '+project] = []

        model_path = os.path.join(save_base_dir, f"{model}_project_{project}.joblib")
        if not os.path.exists(model_path):
            continue
        print(cancer, project)
        
        # Find matching fold
        matching_fold = next((f for f in loocv_folds if str(f["project"]) == project), None)
        if matching_fold is None:
            print(f"No matching fold for {project_id}")
            
        train_idx = matching_fold["train_idx"]
        val_idx = matching_fold["val_idx"]
        
        use_sparse = supports_sparse.get(model, False)
        
        X_val=X[val_idx] if use_sparse else X[val_idx].toarray()
        
        print('Loading ', model_path)

        # Load model
        best_model = joblib.load(model_path)
                
        y_true = y_3class[val_idx]
        y_pred = best_model.predict(X_val)
        y_prob = best_model.predict_proba(X_val)

        ad_tmp = ad[val_idx]
        ad_tmp.raw = None
        ad_tmp.obs['pred_prob_0'] = y_prob[:, 0]
        ad_tmp.obs['pred_prob_1'] = y_prob[:, 1]
        ad_tmp.obs['pred_prob_2'] = y_prob[:, 2]

        ad_tmp.obs['pred_label'] = y_pred
        all_results_ad[model][cancer+' '+project] = ad_tmp


*************** LogisticRegression ***************
Lung 2096-Lungcancer
Loading  /scratch/gilbreth/wang3712/Metastasis_single_cell/Baseline_LOOCV/LogisticRegression_project_2096-Lungcancer.joblib
Colorectal 2098-Colorectalcancer
Loading  /scratch/gilbreth/wang3712/Metastasis_single_cell/Baseline_LOOCV/LogisticRegression_project_2098-Colorectalcancer.joblib
Ovarian 2100-Ovariancancer
Loading  /scratch/gilbreth/wang3712/Metastasis_single_cell/Baseline_LOOCV/LogisticRegression_project_2100-Ovariancancer.joblib
Breast 2102-Breastcancer
Loading  /scratch/gilbreth/wang3712/Metastasis_single_cell/Baseline_LOOCV/LogisticRegression_project_2102-Breastcancer.joblib
Lung Chen_Zhang_2020
Loading  /scratch/gilbreth/wang3712/Metastasis_single_cell/Baseline_LOOCV/LogisticRegression_project_Chen_Zhang_2020.joblib
Colorectal GSE132465
Loading  /scratch/gilbreth/wang3712/Metastasis_single_cell/Baseline_LOOCV/LogisticRegression_project_GSE132465.joblib
Breast GSE161529
Loading  /scratch/gilbreth/wang3712

In [51]:
all_results_summary = dict()
for model in all_results_ad.keys():
    print("*" * 15, model, "*" * 15)

    all_results_summary[model] = dict()
    for project in all_results_ad[model].keys():
        print(project)
        # continue
        project_ad =  all_results_ad[model][project]
        all_patients =project_ad.obs['Final_sample_id'].unique()
        for patient in all_patients:
            all_results_summary[model][patient] = dict()
            patient_ad = project_ad[project_ad.obs['Final_sample_id']==patient]
            if patient_ad.n_obs <= 50:
                continue
            # print(patient_ad)
            res = patient_ad.obs['pred_label'].value_counts()
            # print(res)
            all_results_summary[model][patient]['Patient ID'] = patient
            
            try:
                all_results_summary[model][patient]['Primary'] = res.loc[0]/patient_ad.shape[0]*100
            except:
                all_results_summary[model][patient]['Primary'] = 0
            try:
                all_results_summary[model][patient]['Local'] = res.loc[1]/patient_ad.shape[0]*100
            except:
                all_results_summary[model][patient]['Local'] = 0
            try:    
                all_results_summary[model][patient]['Distant'] = res.loc[2]/patient_ad.shape[0]*100
            except:
                all_results_summary[model][patient]['Distant'] = 0 
                
            all_results_summary[model][patient]['Total Num'] = patient_ad.shape[0]
            all_results_summary[model][patient]['True Label'] = patient_ad.obs['source'].loc[patient_ad.obs.index].unique()[0]

            all_results_summary[model][patient]['Project'] = project.split(' ')[-1]
            all_results_summary[model][patient]['Cancer'] = project.split(' ')[0]
            # break
        # break

*************** LogisticRegression ***************
Lung 2096-Lungcancer
Colorectal 2098-Colorectalcancer
Ovarian 2100-Ovariancancer
Breast 2102-Breastcancer
Lung Chen_Zhang_2020
Colorectal GSE132465
Breast GSE161529
Breast GSE167036
Ovarian GSE173682
Colorectal GSE178341
Breast GSE225600
Colorectal GSE225857
Lung Goveia_Carmeliet_2020
Lung HTAN_MSK_SCLC
Lung He_Fan_2021
Lung Kim_Lee_2020
Lung Lambrechts_Thienpont_2018
Lung Laughney_Massague_2020
Lung Leader_Merad_2021
Ovarian MSK_SPECTRUM
Lung Maynard_Bivona_2020
Breast Multi_modal_breast_cancer
Breast Single_cell_map_anti-PD1
Lung UKIM-V
Breast Wu_etal_2021_BRCA
Ovarian Zhang_2019
Lung Zilionis_Klein_2019
*************** SGD ***************
Lung 2096-Lungcancer
Colorectal 2098-Colorectalcancer
Ovarian 2100-Ovariancancer
Breast 2102-Breastcancer
Lung Chen_Zhang_2020
Colorectal GSE132465
Breast GSE161529
Breast GSE167036
Ovarian GSE173682
Colorectal GSE178341
Breast GSE225600
Colorectal GSE225857
Lung Goveia_Carmeliet_2020
Lung HTAN_MSK

This is what's reported in the paper.

In [52]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score
import numpy as np
import pandas as pd

for model in all_results_summary.keys():
    print("*" * 15, model, "*" * 15)
    
    model_res_df = pd.DataFrame.from_dict(all_results_summary[model], orient='index')

    # Define prediction logic
    cond1 = (model_res_df["Primary"] > model_res_df["Local"]) & (model_res_df["Primary"] > model_res_df["Distant"])
    cond2 = ~cond1
    # cond3 =  & ~cond2

    choices = ["Primary", "local"]
    model_res_df["Predicted_Label"] = np.select([cond1, cond2], choices)
    model_res_df["Correct"] = model_res_df["Predicted_Label"] == model_res_df["True Label"]

    # Binary format for metrics
    y_true_bin = model_res_df["True Label"].map({"Primary": 1, "local": 0})
    y_pred_bin = model_res_df["Predicted_Label"].map({"Primary": 1, "local": 0})
    y_scores = model_res_df["Primary"]  # Score for Primary class

    metrics_list = []

    for project, df_proj in model_res_df.groupby("Project"):
        y_true = df_proj["True Label"].map({"Primary": 1, "local": 0})
        y_pred = df_proj["Predicted_Label"].map({"Primary": 1, "local": 0})
        y_score = df_proj["Primary"]

        try:
            auc = roc_auc_score(y_true, y_score)
        except:
            auc = np.nan
        try:
            auprc = average_precision_score(y_true, y_score)
        except:
            auprc = np.nan

        metrics_list.append({
            "Project": project,
            "Accuracy": accuracy_score(y_true, y_pred),
            "F1": f1_score(y_true, y_pred),
            "AUROC": auc,
            "AUPRC": auprc
        })

    metrics_df = pd.DataFrame(metrics_list)

    # Print summary
    print(f"Accuracy: {metrics_df['Accuracy'].mean():.4f} ± {metrics_df['Accuracy'].std():.4f}")
    print(f"F1 Score: {metrics_df['F1'].mean():.4f} ± {metrics_df['F1'].std():.4f}")
    print(f"AUROC:    {metrics_df['AUROC'].mean():.4f} ± {metrics_df['AUROC'].std():.4f}")
    print(f"AUPRC:    {metrics_df['AUPRC'].mean():.4f} ± {metrics_df['AUPRC'].std():.4f}")
    print()


*************** LogisticRegression ***************
Accuracy: 0.6214 ± 0.3668
F1 Score: 0.6223 ± 0.4176
AUROC:    0.6535 ± 0.3043
AUPRC:    0.7537 ± 0.3874

*************** SGD ***************
Accuracy: 0.5983 ± 0.3685
F1 Score: 0.6121 ± 0.4135
AUROC:    0.6418 ± 0.2995
AUPRC:    0.7542 ± 0.3874

*************** RandomForest ***************


/home/wang3712/.conda/envs/cent7/2020.11-py38/gears/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:979: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/wang3712/.conda/envs/cent7/2020.11-py38/gears/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:979: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/wang3712/.conda/envs/cent7/2020.11-py38/gears/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:979: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/wang3712/.conda/envs/cent7/2020.11-py38/gears/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:979: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/wang3712/.conda/envs/cent7/2020.11-py38/gears/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:979: UserWarning

Accuracy: 0.6215 ± 0.3812
F1 Score: 0.6647 ± 0.3958
AUROC:    0.4192 ± 0.2188
AUPRC:    0.6951 ± 0.3836



/home/wang3712/.conda/envs/cent7/2020.11-py38/gears/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:979: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/wang3712/.conda/envs/cent7/2020.11-py38/gears/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:979: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/wang3712/.conda/envs/cent7/2020.11-py38/gears/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:979: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/wang3712/.conda/envs/cent7/2020.11-py38/gears/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:979: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


# 5 Fold CV

## Data preperation

In [ ]:

# === Extract Features (dense numpy) ===
X = torch.tensor(ad.X.toarray(), dtype=torch.float32) if not isinstance(ad.X, np.ndarray) else torch.tensor(ad.X, dtype=torch.float32)

# === Define Class Labels ===
primary_mask = (ad.obs["Primary_or_Metastatic"] == "Primary").values
local_mask = (ad.obs["Primary_or_Metastatic"] == "Metastatic") & (ad.obs["source"] == "local").values
distant_mask = (ad.obs["Primary_or_Metastatic"] == "Metastatic") & (ad.obs["source"] == "distant").values


In [ ]:
from sklearn.model_selection import StratifiedKFold

# Multi-class: 0 = Primary, 1 = Local, 2 = Distant
y_3class = np.full(X.shape[0], -1, dtype=int)
y_3class[primary_mask] = 0
y_3class[local_mask] = 1
y_3class[distant_mask] = 2
y_3class = torch.tensor(y_3class, dtype=torch.long)

# Binary: 0 = Primary, 1 = Local, -1 = Distant (for eval only)
y_binary = np.full(X.shape[0], -1, dtype=int)
y_binary[primary_mask] = 0
y_binary[local_mask] = 1
y_binary = torch.tensor(y_binary, dtype=torch.long)

# === Valid mask for stratification (only Primary and Local mets) ===
valid_mask = (y_3class == 0) | (y_3class == 1)
valid_idx = np.where(valid_mask.numpy())[0]

# === Stratified 5-Fold CV ===
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds = []
for train_idx, val_idx in skf.split(valid_idx, y_3class[valid_mask]):
    # Store indices with respect to the full dataset (not just valid subset)
    folds.append({
        "train_idx": valid_idx[train_idx],
        "val_idx": valid_idx[val_idx]
    })

# === Export for classical models ===
X_all = X.numpy()
y_3class_np = y_3class.numpy()
y_binary_np = y_binary.numpy()
cv_folds = folds

print(f"Data prepared. Total cells: {X_all.shape[0]}")
print(f"Primary: {(y_3class_np == 0).sum().item()}, Local: {(y_3class_np == 1).sum().item()}, Distant: {(y_3class_np == 2).sum().item()}")


In [16]:
# free the memory
memory_usgae()
del X, ad
memory_usgae()

Current memory usage: 12.99 GB
Current memory usage: 12.99 GB


## Run Models

In [17]:
def evaluate_baseline_models(model, X_all, y_3class, y_binary, folds):
    """
    Train on full y_3class (0=Primary, 1=Local Mets, 2=Distant Mets),
    but evaluate on binary subset (Primary vs Local only).
    """
    accs, f1s, aucs, auprcs = [], [], [], []

    for fold_num, fold in enumerate(folds):
        train_idx = fold["train_idx"]
        val_idx = fold["val_idx"]

        # === Train on all 3 classes ===
        X_train = X_all[train_idx]
        y_train = y_3class[train_idx]  # contains 0, 1, 2

        # === Validation set contains Primary + Local + Distant ===
        X_val = X_all[val_idx]
        y_val_3class = y_3class[val_idx]     # 0, 1, or 2
        y_val_binary = y_binary[val_idx]     # -1 for Distant

        # === Fit the model ===
        model.fit(X_train, y_train)

        # === Predict ===
        y_pred_all = model.predict(X_val)
        if hasattr(model, "predict_proba"):
            y_proba_all = model.predict_proba(X_val)
        else:
            y_proba_all = None

        # === Filter for binary evaluation only (Primary vs Local) ===
        mask = (y_val_binary != -1)
        y_true = y_val_binary[mask]
        y_pred = y_pred_all[mask]
        y_prob = y_proba_all[mask, 1] if y_proba_all is not None else y_pred

        # === Metrics ===
        acc = accuracy_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred, average='weighted')
        try:
            auc = roc_auc_score(y_true, y_prob)
        except:
            auc = np.nan
        try:
            auprc = average_precision_score(y_true, y_prob)
        except:
            auprc = np.nan

        accs.append(acc)
        f1s.append(f1)
        aucs.append(auc)
        auprcs.append(auprc)

        print(f"Fold {fold_num+1}: ACC = {acc:.4f}, F1 = {f1:.4f}, AUC = {auc:.4f}, AUPRC = {auprc:.4f}")

    print("Overall Performance (Non-metastatic local vs Metastatic local):")
    print(f"ACC:   {np.mean(accs):.4f} ± {np.std(accs):.4f}")
    print(f"F1:    {np.mean(f1s):.4f} ± {np.std(f1s):.4f}")
    print(f"AUC:   {np.mean(aucs):.4f} ± {np.std(aucs):.4f}")
    print(f"AUPRC: {np.mean(auprcs):.4f} ± {np.std(auprcs):.4f}")

    return {
        "acc": accs,
        "f1": f1s,
        "auc": aucs,
        "auprc": auprcs
    }


In [18]:
# run it seperately, as it takes too long
model_name = list(models.keys())[0]
model = models[model_name]
print(f"\nEvaluating Model: {model_name}")
results = evaluate_baseline_models( 
    model=model,
    X_all=X_all,
    y_3class=y_3class_np,
    y_binary=y_binary_np,
    folds=cv_folds
)


Evaluating Model: LogisticRegression
Fold 1: ACC = 0.9219, F1 = 0.9212, AUC = 0.9694, AUPRC = 0.9474
Fold 2: ACC = 0.9213, F1 = 0.9206, AUC = 0.9683, AUPRC = 0.9462
Fold 3: ACC = 0.9213, F1 = 0.9208, AUC = 0.9687, AUPRC = 0.9466
Fold 4: ACC = 0.9200, F1 = 0.9194, AUC = 0.9689, AUPRC = 0.9471
Fold 5: ACC = 0.9217, F1 = 0.9210, AUC = 0.9687, AUPRC = 0.9473
Overall Performance (Non-metastatic local vs Metastatic local):
ACC:   0.9212 ± 0.0007
F1:    0.9206 ± 0.0006
AUC:   0.9688 ± 0.0003
AUPRC: 0.9469 ± 0.0005


In [19]:
# run it seperately, as it takes too long
model_name = list(models.keys())[1]
model = models[model_name]
print(f"\nEvaluating Model: {model_name}")
results = evaluate_baseline_models(
    model=model,
    X_all=X_all,
    y_3class=y_3class_np,
    y_binary=y_binary_np,
    folds=cv_folds
)


Evaluating Model: RandomForest
Fold 1: ACC = 0.9275, F1 = 0.9248, AUC = 0.9835, AUPRC = 0.9718
Fold 2: ACC = 0.9262, F1 = 0.9234, AUC = 0.9837, AUPRC = 0.9721
Fold 3: ACC = 0.9276, F1 = 0.9249, AUC = 0.9842, AUPRC = 0.9729
Fold 4: ACC = 0.9274, F1 = 0.9247, AUC = 0.9840, AUPRC = 0.9725
Fold 5: ACC = 0.9255, F1 = 0.9226, AUC = 0.9840, AUPRC = 0.9725
Overall Performance (Non-metastatic local vs Metastatic local):
ACC:   0.9268 ± 0.0008
F1:    0.9241 ± 0.0009
AUC:   0.9839 ± 0.0003
AUPRC: 0.9724 ± 0.0004


In [20]:
# run it seperately, as it takes too long
model_name = list(models.keys())[3]
model = models[model_name]
print(f"\nEvaluating Model: {model_name}")
results = evaluate_baseline_models(
    model=model,
    X_all=X_all,
    y_3class=y_3class_np,
    y_binary=y_binary_np,
    folds=cv_folds
)


Evaluating Model: SGD
Fold 1: ACC = 0.9161, F1 = 0.9155, AUC = 0.8948, AUPRC = 0.7930
Fold 2: ACC = 0.9144, F1 = 0.9127, AUC = 0.8828, AUPRC = 0.7936
Fold 3: ACC = 0.9047, F1 = 0.9016, AUC = 0.8628, AUPRC = 0.7750
Fold 4: ACC = 0.9152, F1 = 0.9147, AUC = 0.8951, AUPRC = 0.7904
Fold 5: ACC = 0.8912, F1 = 0.8933, AUC = 0.8966, AUPRC = 0.7360
Overall Performance (Non-metastatic local vs Metastatic local):
ACC:   0.9083 ± 0.0095
F1:    0.9076 ± 0.0087
AUC:   0.8864 ± 0.0128
AUPRC: 0.7776 ± 0.0219
